# DS20: Richter's Predictor: Modeling Earthquake Damage

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from category_encoders import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

# Wrangle Data

## Import Data

In [ ]:
dtypes = {'geo_level_1_id': 'object'}#,
          #'geo_level_2_id': 'object',
          #'geo_level_3_id': 'object'}

df = pd.merge(pd.read_csv('data/train_values.csv', dtype=dtypes),
              pd.read_csv('data/train_labels.csv')).set_index('building_id')

## EDA

- One possible problem: `NaN`'s masquerading as `0`s.

In [ ]:
df.head()

In [ ]:
df.select_dtypes('object').nunique()

In [ ]:
df.info()

# Split Data

In [ ]:
target = 'damage_grade'
X = df.drop(columns=target)
y = df[target]

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Establish Baseline

In [ ]:
print('Baseline Accuracy:', y_train.value_counts(normalize=True).max())

# Build Model

In [ ]:
model = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    RandomForestClassifier(random_state=42, n_jobs=-1)
)

In [ ]:
model.fit(X_train, y_train);

# Check Metrics

Accuracy

In [ ]:
print('Training Accuracy:', accuracy_score(y_train, model.predict(X_train)))
print('Validation Accuracy:', accuracy_score(y_val, model.predict(X_val)))

In [ ]:
print('Training F1:', f1_score(y_train, model.predict(X_train), average='micro'))
print('Validation F1:', f1_score(y_val, model.predict(X_val), average='micro'))

# Make Submission

In [ ]:
X_test = pd.read_csv('data/test_values.csv', dtype=dtypes, index_col='building_id')
X_test.head()

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred[:10]

In [ ]:
submission = pd.DataFrame({'damage_grade':y_pred}, index=X_test.index)
submission.head()

In [ ]:
pd.Timestamp.now().strftime('%Y-%m-%d_%H-%M')

In [ ]:
timestamp = pd.Timestamp.now().strftime('%Y-%m-%d_%H-%M')
submission.to_csv(f'submissions/{timestamp}_predictions.csv', index=True)